# Comparison ATL03, ATL08, ATL12, and ATL13

## Data downloaded with SlideRule

## Computing environment

We'll be using the following Python libraries in this notebook:

In [1]:
import numpy as np
import time
from datetime import datetime, timedelta
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from pyproj import Transformer, CRS
from shapely.geometry import Polygon, Point
from sliderule import sliderule
from sliderule import icesat2
from pyproj import Transformer, CRS
import requests

    
The import below is a class that the organizers wrote. It helps us read and store data from the OpenAltimetry API.  This line is importing the class `dataCollector` in the file `oa.py` in the `utils` folder.

In [2]:
url="icesat2sliderule.org"
icesat2.init(url, verbose=False)

## Define function to get data

In [3]:
def ATL03_data_for_granule(granule, pair, poly, params=None, epsg=3857, asset="nsidc-s3", beam='strong'):
    if params is None:
        params={'poly':poly, 'srt':0, 'cnf':0}
    else:
        params.update({'poly':poly})
    
    gdf = gpd.GeoDataFrame(icesat2.atl03s(params, granule.replace('ATL06','ATL03'), track=int(pair), asset=asset))
    crs=CRS.from_epsg(epsg)
    to_xy_crs=Transformer.from_crs(crs.geodetic_crs, crs)
    x, y, z = to_xy_crs.transform(gdf['geometry'].y.values, gdf['geometry'].x.values, gdf['height'])
    gdf['x'] = x
    gdf['y'] = y
    gdf['height'] = z
    
    #tested on cycle 11, where the strong beam in the pair matches the spacecraft orientation.  Need to check 
    # on other cycles
    if beam=='strong':
        gdf=gdf[gdf['sc_orient'] == gdf['pair']]
    elif beam=='weak':
        gdf=gdf[gdf['sc_orient'] != gdf['pair']]
    # otherwise, return both beams
    return gdf

## Get data URL from the OpenAltimetry API

* Go to [openaltimetry.org](https://openaltimetry.org/)
* Select **BROWSE ICESAT-2 DATA**. 
* Select **ATL 08** on the left
* Select a date
* Click **SELECT A REGION** on the top left, and drew a rectangle
* Right click on the rectangle and select **View Elevation profile**. This opens a pop up, and shows ATL06 and ATL08 elevations. 
* Scroll to the **bottom** and select **Get API URL**. Copy it.

We can now initialize a `dataCollector` object, using the copy-pasted OpenAltimetry API URL, and the beam we would like to look at. 

Note, we don't need EarthData credentials for this.

In [ ]:
url = 'http://openaltimetry.org/data/api/icesat2/atl13?date=2020-12-15&minx=-77.858681&miny=25.728091&maxx=-77.831461&maxy=25.832559&trackId=1254&beamName=gt1r&beamName=gt1l&outputFormat=json'
gtx = 'gt1r'
is2data = dataCollector(oaurl=url, beam=gtx, verbose=True)

### Another way to get the OA URL


Alternatively, we could use a date, track number, beam, and lat/lon bounding box as input to the `dataCollector`.

In [ ]:
latlims = [-70.34957, -70.24491]
lonlims = [12.10769, 12.42636]
rgt = 312 # track number
gtx = 'gt2r' # The beam
date = '2020-01-16'
is2data = dataCollector(date=date, latlims=latlims, lonlims=lonlims, track=rgt, beam=gtx, verbose=True)

Note that this also constructs the API url for us.

### Requesting the data from the OpenAltimetry API
Here we use the `requestData()` function of the `dataCollector` class, which is defined in `utils/oa.py`. It downloads ATL03, ATL06 and ATL08 data based on the inputs with which we initialized our `dataCollector`, and writes them to pandas dataframes.

In [ ]:
is2data.requestData(verbose=True)

The data are now stored as pandas data frames in our `dataCollector` object. To verify this, we can run the cell below.

## Plotting the data

To make plots easier to produce, the `dataCollector` class in `utils/oa.py` has a method to plot the data that we downloaded. 

In [ ]:
%%capture
fig = is2data.plotData()

We can make a function to put all this together.

In [ ]:
def plot_from_oa_url(url,gtx,title='ICESat-2 Data'):
    mydata = dataCollector(oaurl=url,beam=gtx)
    mydata.requestData()
    myplot = mydata.plotData(title=title)
    return (myplot, mydata)

## Examples

In [ ]:
%%capture
url = 'http://openaltimetry.org/data/api/icesat2/atl13?date=2020-12-15&minx=-77.858681&miny=25.728091&maxx=-77.831461&maxy=25.832559&trackId=1254&beamName=gt1r&beamName=gt1l&outputFormat=json'
gtx = 'gt1r'
myplot, mydata = plot_from_oa_url(url=url, gtx=gtx, title='Coastal Example')
myplot.savefig('geemap_tutorial_exercise1.jpg', dpi=300)

In [ ]:
myplot

In [ ]:
%%capture
#use capture to prevent the plot
url = 'http://openaltimetry.org/data/api/icesat2/atl12?date=2021-12-15&minx=-154.32381980236076&miny=56.873762230836434&maxx=-154.29895036037468&maxy=56.93202651616244&trackId=1280&beamName=gt1r&beamName=gt1l&outputFormat=json'
gtx = 'gt1r'
myplot, mydata = plot_from_oa_url(url=url, gtx=gtx, title='Alaska Coast')
myplot.savefig('geemap_tutorial_exercise1.jpg', dpi=300)

In [ ]:
myplot

In [ ]:
%%capture
url = 'http://openaltimetry.org/data/api/icesat2/atl03?date=2020-01-16&minx=12.107692195781404&miny=-70.34956862465471&maxx=12.426364789894341&maxy=-70.2449105354736&trackId=312&beamName=gt3r&beamName=gt3l&beamName=gt2r&beamName=gt2l&beamName=gt1r&beamName=gt1l&outputFormat=json'
gtx = 'gt2r'
myplot, mydata = plot_from_oa_url(url=url, gtx=gtx, title='Lake Example')
myplot.savefig('geemap_tutorial_exercise1.jpg', dpi=300)

In [ ]:
myplot